<a href="https://colab.research.google.com/github/SanjayJanardhan-89/ApacheSparkHandsOn/blob/main/ArrayFunctions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup Pyspark


In [1]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"


import findspark
findspark.init()
findspark.find()

import pyspark
from pyspark.sql import SparkSession

spark= SparkSession \
       .builder \
       .appName("OurSparkApp") \
       .getOrCreate()

spark

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,679 kB]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,767 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,692 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,237 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,535 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/

# Create Test data

In [48]:
from pyspark.sql.functions import collect_list, collect_set,array_contains,array_distinct \
, array_except,array_intersect \
, array_join, array_max,array_min, explode, slice

In [13]:


data = [
    ("x", 4, 1),
    ("x", 6, 2),
    ("z", 7, 3),
    ("a", 3, 4),
    ("z", 5, 2),
    ("x", 7, 3),
    ("x", 9, 7),
    ("z", 1, 8),
    ("z", 4, 9),
    ("z", 7, 4),
    ("a", 8, 5),
    ("a", 5, 2),
    ("a", 3, 8),
    ("x", 2, 7),
    ("z", 1, 9)
]

columns = ["col1", "col2", "col3"]
initial_df = spark.createDataFrame(data, columns)

# Generate Array columns
full_df = (initial_df
            .groupBy("col1")
              .agg(
                  collect_list("col2").alias("array_col1"),
                  collect_list("col3").alias("array_col2")
                  )
          )

# Drop the "array_col1" column
df = full_df.drop("array_col1")

# Show result
df.show()
df.printSchema()

+----+------------------+
|col1|        array_col2|
+----+------------------+
|   x|   [1, 2, 3, 7, 7]|
|   z|[3, 2, 8, 9, 4, 9]|
|   a|      [4, 5, 2, 8]|
+----+------------------+

root
 |-- col1: string (nullable = true)
 |-- array_col2: array (nullable = false)
 |    |-- element: long (containsNull = false)



In [9]:
full_df.show()
full_df.printSchema()

+----+------------------+------------------+
|col1|        array_col1|        array_col2|
+----+------------------+------------------+
|   x|   [4, 6, 7, 9, 2]|   [1, 2, 3, 7, 7]|
|   z|[7, 5, 1, 4, 7, 1]|[3, 2, 8, 9, 4, 9]|
|   a|      [3, 8, 5, 3]|      [4, 5, 2, 8]|
+----+------------------+------------------+

root
 |-- col1: string (nullable = true)
 |-- array_col1: array (nullable = false)
 |    |-- element: long (containsNull = false)
 |-- array_col2: array (nullable = false)
 |    |-- element: long (containsNull = false)



# Array Functions


In [15]:
df.withColumn("result", array_contains("array_col2", 3)).show()

+----+------------------+------+
|col1|        array_col2|result|
+----+------------------+------+
|   x|   [1, 2, 3, 7, 7]|  true|
|   z|[3, 2, 8, 9, 4, 9]|  true|
|   a|      [4, 5, 2, 8]| false|
+----+------------------+------+



In [18]:
df.withColumn("result", array_distinct("array_col2")).show()

+----+------------------+---------------+
|col1|        array_col2|         result|
+----+------------------+---------------+
|   x|   [1, 2, 3, 7, 7]|   [1, 2, 3, 7]|
|   z|[3, 2, 8, 9, 4, 9]|[3, 2, 8, 9, 4]|
|   a|      [4, 5, 2, 8]|   [4, 5, 2, 8]|
+----+------------------+---------------+



In [26]:
full_df.withColumn("result", array_except("array_col1", "array_col2")).show()

+----+------------------+------------------+---------+
|col1|        array_col1|        array_col2|   result|
+----+------------------+------------------+---------+
|   x|   [4, 6, 7, 9, 2]|   [1, 2, 3, 7, 7]|[4, 6, 9]|
|   z|[7, 5, 1, 4, 7, 1]|[3, 2, 8, 9, 4, 9]|[7, 5, 1]|
|   a|      [3, 8, 5, 3]|      [4, 5, 2, 8]|      [3]|
+----+------------------+------------------+---------+



In [28]:
full_df.withColumn("result", array_intersect("array_col1", "array_col2")).show()

+----+------------------+------------------+------+
|col1|        array_col1|        array_col2|result|
+----+------------------+------------------+------+
|   x|   [4, 6, 7, 9, 2]|   [1, 2, 3, 7, 7]|[7, 2]|
|   z|[7, 5, 1, 4, 7, 1]|[3, 2, 8, 9, 4, 9]|   [4]|
|   a|      [3, 8, 5, 3]|      [4, 5, 2, 8]|[8, 5]|
+----+------------------+------------------+------+



In [32]:
df.withColumn("result", array_join("array_col2", ",")).show()

+----+------------------+-----------+
|col1|        array_col2|     result|
+----+------------------+-----------+
|   x|   [1, 2, 3, 7, 7]|  1,2,3,7,7|
|   z|[3, 2, 8, 9, 4, 9]|3,2,8,9,4,9|
|   a|      [4, 5, 2, 8]|    4,5,2,8|
+----+------------------+-----------+



In [38]:
(df
  .withColumn("result-max", array_max("array_col2"))
  .withColumn("result-min", array_min("array_col2"))
  .withColumn("result-min", array_min("array_col2"))
  .show()
)

+----+------------------+----------+----------+
|col1|        array_col2|result-max|result-min|
+----+------------------+----------+----------+
|   x|   [1, 2, 3, 7, 7]|         7|         1|
|   z|[3, 2, 8, 9, 4, 9]|         9|         2|
|   a|      [4, 5, 2, 8]|         8|         2|
+----+------------------+----------+----------+



In [45]:
(df
  .withColumn("slice result", slice("array_col2", 1,2))
  .withColumn("exploded result", explode("slice result"))
  .show()
)

+----+------------------+------------+---------------+
|col1|        array_col2|slice result|exploded result|
+----+------------------+------------+---------------+
|   x|   [1, 2, 3, 7, 7]|      [1, 2]|              1|
|   x|   [1, 2, 3, 7, 7]|      [1, 2]|              2|
|   z|[3, 2, 8, 9, 4, 9]|      [3, 2]|              3|
|   z|[3, 2, 8, 9, 4, 9]|      [3, 2]|              2|
|   a|      [4, 5, 2, 8]|      [4, 5]|              4|
|   a|      [4, 5, 2, 8]|      [4, 5]|              5|
+----+------------------+------------+---------------+

